In [1]:
import numpy as np
import laser_hockey_env as lh
import gym
from importlib import reload

In [8]:
np.set_printoptions(suppress=True)

# Normal Game Play

In [13]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-env/laser_hockey_env.py'>

In [16]:
env = lh.LaserHockeyEnvContinuous()

have a look at the initialization condition

In [20]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()
env.render()

True

In [17]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):
    env.render()
    a1 = [1,-.5,0] # np.random.uniform(-1,1,3)
    a2 = [1,0.,0] # np.random.uniform(-1,1,3)*0    
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

Without rendering, it runs much faster

In [19]:
obs = env.reset()
obs_agent2 = env.obs_agent_two()

for _ in range(600):    
    a1 = [1,-.5,0] # np.random.uniform(-1,1,3)
    a2 = [1,0.,0] # np.random.uniform(-1,1,3)*0    
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Train Shooting

In [12]:
reload(lh)

<module 'laser_hockey_env' from '/home/georg/src/python/laser-hockey-env/laser_hockey_env.py'>

In [232]:
env = lh.LaserHockeyEnvContinuous(mode=lh.LaserHockeyEnv.TRAIN_SHOOTING)

In [240]:
o = env.reset()

In [241]:
env.render()

True

In [243]:
for _ in range(60):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break

# Train DEFENDING

In [200]:
reload(lh)

<module 'envs.laser_hockey_env' from '/home/georg/src/python/laser-hockey-rl/envs/laser_hockey_env.py'>

In [221]:
env = lh.LaserHockeyEnvContinuous(mode=lh.LaserHockeyEnv.TRAIN_DEFENCE)

In [222]:
o = env.reset()

In [223]:
env.render()

True

In [225]:
for _ in range(60):
    env.render()
    a1 = [1,0,0] # np.random.uniform(-1,1,3)
    a2 = [0,0.,0] 
    obs, r, d, info = env.step(np.hstack([a1,a2]))    
    obs_agent2 = env.obs_agent_two()
    if d: break